# 주식/코인 가격 생성기 - Google Colab

이 노트북은 Google Colab에서 GPU를 사용하여 주식/코인 가격 예측 모델을 학습하고 사용하는 방법을 보여줍니다.

## 🚀 빠른 시작

1. **GPU 활성화**: 런타임 → 런타임 유형 변경 → GPU
2. 아래 셀들을 순서대로 실행하세요

## 1. 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers pandas numpy yfinance requests accelerate python-dotenv

## 2.5 API 키 설정 (뉴스 API 사용 시, 선택)

**.env를 이미 2번에서 업로드했다면 이 셀은 건너뛰세요.**

- **A) .env 업로드**: 2번 `files.upload()` 때 `.env`도 선택 → 추가 작업 없음
- **B) 여기서 .env 만들기**: 아래 코드 셀 실행 후 `getpass`로 키 입력 (노트북에 안 보임)
- **C) 환경 변수**: `os.environ['NEWSAPI_KEY']='키'` 설정 (노트북에 키 노출됨, 비공개일 때만)

In [ ]:
# [선택] .env를 업로드 안 했을 때만 아래 중 하나를 사용하세요.
# .env 형식: NEWSAPI_KEY=키, ALPHAVANTAGE_API_KEY=키, FINNHUB_API_KEY=키 (이름 정확히)

# 방법 B: getpass로 .env 생성 (키가 화면에 안 보임) → Colab /content 에 저장 후 로드
# from getpass import getpass
# import os
# lines = []
# for n in ['NEWSAPI_KEY', 'ALPHAVANTAGE_API_KEY', 'FINNHUB_API_KEY']:
#     v = getpass(f'{n} (엔터=건너뛰기): ')
#     if (v or '').strip():
#         lines.append(f"{n}={v.strip()}\n")
# if lines:
#     with open('.env', 'w') as f:
#         f.writelines(lines)
#     from dotenv import load_dotenv
#     load_dotenv('.env')
#     load_dotenv('/content/.env')
#     print('✓ .env 생성 및 로드 완료')

# 방법 C: 환경 변수 (노트북에 키 노출 → 비공개일 때만)
# import os
# os.environ['NEWSAPI_KEY'] = 'your_key_here'
# os.environ['ALPHAVANTAGE_API_KEY'] = '...'   # 선택
# os.environ['FINNHUB_API_KEY'] = '...'       # 선택

# .env 업로드했거나 yfinance만 쓸 경우 → 이 셀 건너뛰기

## 2. 파일 업로드

In [ ]:
# go_stock.py (필수) + .env (뉴스 API 사용 시) 업로드
from google.colab import files
uploaded = files.upload()  # .env도 같이 선택하면 뉴스 API 키 사용 가능

import os

if 'go_stock.py' in os.listdir('.'):
    print("✓ go_stock.py 업로드 완료!")
else:
    print("⚠ go_stock.py를 업로드해주세요.")

# .env 탐지: Colab은 업로드 파일이 cwd 또는 /content에 있음. 숨김파일은 listdir에 안 나올 수 있으므로 os.path.isfile 사용
_cwd = os.getcwd()
_env_candidates = ['/content/.env', os.path.join(_cwd, '.env'), '.env']
_env_path = None
for p in _env_candidates:
    if p and os.path.isfile(p):
        _env_path = os.path.abspath(p) if not os.path.isabs(p) else p
        break
if _env_path is None and isinstance(uploaded, dict) and '.env' in uploaded:
    # 업로드했는데 아직 디스크에 안 쓰였을 수 있음 → cwd 기준 한 번 더
    _env_path = os.path.abspath('.env') if os.path.isfile('.env') else os.path.join(_cwd, '.env')

if _env_path and os.path.isfile(_env_path):
    print("✓ .env 발견:", _env_path)
    try:
        from dotenv import load_dotenv
        load_dotenv(_env_path)
        load_dotenv('/content/.env')
        load_dotenv(os.path.join(_cwd, '.env'))
        # load_dotenv가 BOM/인코딩 때문에 실패할 수 있음 → 직접 파싱 (로컬 check_news와 동일)
        _want = ('NEWSAPI_KEY', 'ALPHAVANTAGE_API_KEY', 'FINNHUB_API_KEY')
        for _p in [_env_path, '/content/.env', os.path.join(_cwd, '.env')]:
            if not _p or not os.path.isfile(_p):
                continue
            try:
                with open(_p, 'r', encoding='utf-8-sig') as _f:
                    for _line in _f:
                        _line = _line.strip()
                        if not _line or _line.startswith('#') or '=' not in _line:
                            continue
                        _k, _v = _line.split('=', 1)
                        _k, _v = _k.strip(), _v.strip().strip('"').strip("'")
                        if _k in _want and not os.getenv(_k) and _v:
                            os.environ[_k] = _v
            except Exception:
                pass
        def _m(s):
            return '로드됨' if (s and len(str(s).strip()) >= 4) else '없음'
        for k in ('NEWSAPI_KEY', 'ALPHAVANTAGE_API_KEY', 'FINNHUB_API_KEY'):
            print(f"   {k}: {_m(os.getenv(k, ''))}")
        if not any(os.getenv(k) for k in ('NEWSAPI_KEY', 'ALPHAVANTAGE_API_KEY', 'FINNHUB_API_KEY')):
            print("   → 전부 없음: .env에 NEWSAPI_KEY=키, ALPHAVANTAGE_API_KEY=키, FINNHUB_API_KEY=키 (한 줄씩, 이름 정확히) 있는지 확인")
    except Exception as e:
        print("  ⚠ .env 로드 오류:", e)
else:
    print("💡 .env 없음. 뉴스 API: 위 'API 키 설정' 셀을 실행하거나 .env를 업로드하세요.")
    if isinstance(uploaded, dict) and uploaded:
        print("   업로드된 파일:", list(uploaded.keys()))

## 3. GPU 확인 및 모델 선택

In [ ]:
import torch
from go_stock import StockPriceGenerator

# GPU 확인
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"  메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    
    # Fin-E5 사용 가능 여부 확인
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    if gpu_memory >= 16:
        print("\n✓ Fin-E5 사용 가능! (최고 성능)")
        embedding_model = 'fine5'
        batch_size = 16
    else:
        print("\n⚠ GPU 메모리가 부족합니다. FinBERT를 사용합니다.")
        embedding_model = 'finbert'
        batch_size = 32
else:
    print("⚠ GPU를 사용할 수 없습니다. CPU로 실행합니다.")
    embedding_model = 'finbert'
    batch_size = 16

print(f"\n📝 선택된 모델: {embedding_model}")
print(f"📝 배치 크기: {batch_size}")

# 뉴스 API 키: go_stock이 .env/환경변수에서 로드 (Colab: /content, cwd 지원)
import os
if any(os.getenv(k) for k in ('NEWSAPI_KEY', 'ALPHAVANTAGE_API_KEY', 'FINNHUB_API_KEY')):
    print("✓ 뉴스 API 키 로드됨 (NewsAPI/AlphaVantage/Finnhub 중 사용 가능한 소스 사용)")
else:
    print("💡 뉴스 API: .env 업로드 또는 'API 키 설정' 셀 실행 시 NewsAPI 등 사용 가능. (yfinance는 키 불필요)")

## 4. 생성기 초기화

In [ ]:
# 생성기 초기화
generator = StockPriceGenerator(embedding_model=embedding_model)
print("✓ 생성기 초기화 완료!")

## 5. 데이터 수집

In [ ]:
# 종목 선택 (변경 가능)
symbol = 'BTC-USD'  # 예: 'BTC-USD', 'ETH-USD', 'AAPL', 'TSLA'
period = '1y'       # 데이터 기간
interval = '1h'     # 시간 간격 (1h, 1d, 1wk, 1mo)

# 가격 데이터 수집
print(f"가격 데이터 수집 중: {symbol}...")
price_data = generator.collect_price_data(symbol, period=period, interval=interval)
print(f"✓ 수집 완료: {len(price_data)}개 레코드")

# 뉴스 데이터 수집 (현재는 빈 리스트)
news_data = generator.collect_news_data(symbol, days=365)
print(f"✓ 뉴스 데이터: {len(news_data)}개")

# 데이터 미리보기
print("\n데이터 미리보기:")
print(price_data.head())

## 6. 모델 학습

In [ ]:
# 학습 설정
epochs = 50
learning_rate = 0.001

print(f"학습 시작...")
print(f"  - Epochs: {epochs}")
print(f"  - Batch Size: {batch_size}")
print(f"  - Learning Rate: {learning_rate}")
print(f"  - 모델: {embedding_model}")
print("\n" + "="*60)

# 학습 실행
generator.train(
    price_data, 
    news_data, 
    epochs=epochs, 
    batch_size=batch_size, 
    lr=learning_rate
)

print("\n" + "="*60)
print("✓ 학습 완료!")

## 7. 모델 저장

In [ ]:
# 모델 저장
model_name = f'model_{symbol.replace("-", "_")}_{embedding_model}.pt'
generator.save_model(model_name)
print(f"✓ 모델 저장 완료: {model_name}")

# 모델 다운로드
from google.colab import files
files.download(model_name)
print("✓ 모델 다운로드 시작!")

## 8. 가격 생성 및 시각화

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 최근 가격 데이터 사용
recent_prices = price_data.tail(100)
recent_news = news_data[-10:] if len(news_data) > 0 else []

# 미래 가격 생성
print("미래 가격 생성 중...")
generated_prices = generator.generate_price(recent_prices, recent_news, steps=10)
print(f"✓ 생성 완료: {len(generated_prices)}개 예측값")

# 시각화
plt.figure(figsize=(12, 6))

# 과거 가격
past_dates = pd.to_datetime(recent_prices['datetime']).values[-20:]
past_prices = recent_prices['close'].values[-20:]
plt.plot(past_dates, past_prices, 'b-', label='과거 가격', linewidth=2)

# 생성된 가격
future_dates = pd.date_range(start=past_dates[-1], periods=11, freq='1H')[1:]
plt.plot(future_dates, generated_prices, 'r--', label='예측 가격', linewidth=2, marker='o')

plt.title(f'{symbol} 가격 예측', fontsize=16, fontweight='bold')
plt.xlabel('날짜', fontsize=12)
plt.ylabel('가격 (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 예측값 출력
print("\n생성된 가격:")
for i, price in enumerate(generated_prices, 1):
    print(f"  Step {i}: ${price:.2f}")

## 9. Google Drive 연동 (선택사항)

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 모델을 Drive에 저장
drive_path = f'/content/drive/MyDrive/{model_name}'
generator.save_model(drive_path)
print(f"✓ 모델이 Google Drive에 저장되었습니다: {drive_path}")

# 나중에 로드하려면:
# generator = StockPriceGenerator(model_path=drive_path)

## 💡 팁

- **Fin-E5 사용**: GPU 메모리가 16GB+이면 `embedding_model='fine5'` 사용 (더 높은 정확도)
- **배치 크기 조정**: 메모리 부족 시 `batch_size`를 8 또는 16으로 줄이기
- **데이터 기간 조정**: 메모리 부족 시 `period='6mo'` 또는 `period='3mo'`로 줄이기
- **모델 저장**: 세션이 종료되면 파일이 삭제되므로 반드시 다운로드하거나 Drive에 저장

## 📚 참고 자료

- `GOOGLE_COLAB_SETUP.md`: 상세한 Colab 설정 가이드
- `COLAB_FIN_E5_GUIDE.md`: Fin-E5 사용 가이드
- `TRANSFORMER_MODELS.md`: 모델 비교 가이드